In [1]:
# !pip install wfdb
# !pip install py-ecg-detectors
# !pip install sklearn
# !pip install gensim
# !pip install wget
# !pip install pandas
# !pip install -U scikit-learn

In [36]:
import pickle
import uuid
import os
import wfdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from ecgdetectors import Detectors
from gensim.models import Word2Vec
from datetime import datetime

In [38]:
# Returns (p_waves, qrs_waves, t_waves)
def split_beats_in_p_qrt_t(beats_list):
  c_p_waves = []
  c_qrs_waves = []
  c_t_waves = []
  heart_beat_len = len(beats_list[0])
  retreat_value = heart_beat_len // 2
  for j in range(len(beats_list)):
    c_p_waves.append(beats_list[j][:retreat_value - 15])
    c_qrs_waves.append(beats_list[j][retreat_value - 15:retreat_value + 15])
    c_t_waves.append(beats_list[j][retreat_value + 15:])
  return c_p_waves, c_qrs_waves, c_t_waves

# Returns (kmeans, predicted)
def calculate_kmeans(waves, n_clusters, n_init):
  c_kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=n_init)
  c_kmeans.fit(waves)
  c_predicted = c_kmeans.predict(waves)
  return c_kmeans, c_predicted

# Convert PT cluster predictions to letters
def convert_pt_predictions_to_letters(predictions):
  alphabet_for_pt = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h', 8:'i',
                     9:'j', 10:'k', 11:'l', 12:'m', 13:'n', 14:'o', 15:'p', 16:'q', 17:'r',
                     18:'s', 19:'t'}

  def get_symbol_pt(x):
    return alphabet_for_pt[x]

  vfunc = np.vectorize(get_symbol_pt)

  return vfunc(predictions)

# Convert QRS clusters predictions to letters
def convert_qrs_predictions_to_letters(predictions):
  alphabet_for_qrs = {0:'u', 1:'v', 2:'w', 3:'x', 4:'y', 5:'z'}


  def get_symbol_qrs(x):
    return alphabet_for_qrs[x]

  vfunc_2 = np.vectorize(get_symbol_qrs)

  return vfunc_2(predictions)

def join_waves_letters_to_words(qrs_letters, pt_letters):
  words = []
  signal_half_len = len(qrs_letters)//2
  for i in range(len(qrs_letters)):
    word = ''
    word += pt_letters[i]
    word += qrs_letters[i]
    # T signal is encoded in second half
    word += pt_letters[i + signal_half_len]
    words.append(word)

  return words

def convert_beats_to_words(beats_list):
  (p_waves, qrs, t_waves) = split_beats_in_p_qrt_t(beats_list)
  qrs_kmeans, qrs_predicted = calculate_kmeans(qrs, 6, 3)
  c_pt_waves = np.array(p_waves + t_waves)
  pt_kmeans, pt_predicted = calculate_kmeans(c_pt_waves, 20, 10)

  qrs_letters = convert_qrs_predictions_to_letters(qrs_predicted)
  pt_letters = convert_pt_predictions_to_letters(pt_predicted)
  return join_waves_letters_to_words(qrs_letters, pt_letters)

class BeatsToWordsConverter:
  qrs_kmeans = None
  pt_kmeans = None
  
  def save_to_file(self, file_name):
    pickle.dump(self.qrs_kmeans, open(file_name + "_qrs.pkl", "wb"))
    pickle.dump(self.pt_kmeans, open(file_name + "_pt.pkl", "wb"))
  
  def load_from_file(self, file_name):
    self.qrs_kmeans = pickle.load(open(file_name + "_qrs.pkl", "rb"))
    self.pt_kmeans = pickle.load(open(file_name + "_pt.pkl", "rb"))
  
  def fit_and_predict_words(self, beats_list, cache_file_name, reset_cache=False):
    
    if reset_cache or not (os.path.exists(cache_file_name + "_qrs.pkl") and os.path.exists(cache_file_name + "_pt.pkl")):
      (p_waves, qrs, t_waves) = split_beats_in_p_qrt_t(beats_list)
      print("Fitting QRS KMeans")
      self.qrs_kmeans = KMeans(init='k-means++', n_clusters=6, n_init=3)
      self.qrs_kmeans.fit(qrs)
      
      qrs_predicted = self.qrs_kmeans.predict(qrs)
      
      print("Fitting PT KMeans")
      c_pt_waves = np.array(p_waves + t_waves)
      self.pt_kmeans = KMeans(init='k-means++', n_clusters=20, n_init=10)
      self.pt_kmeans.fit(c_pt_waves)
      
      print("Saving BeatsToWordsConverter to cache: ", cache_file_name)
      self.save_to_file(cache_file_name)
      
      pt_predicted = self.pt_kmeans.predict(c_pt_waves) 
    
      qrs_letters = convert_qrs_predictions_to_letters(qrs_predicted)
      pt_letters = convert_pt_predictions_to_letters(pt_predicted)
      return join_waves_letters_to_words(qrs_letters, pt_letters)
    else:
      print("Using cached BeatsToWordsConverter from file: ", cache_file_name)
      self.load_from_file(cache_file_name)
      return self.predict_words(beats_list)
  
  def predict_words(self, beats_list):
    (p_waves, qrs, t_waves) = split_beats_in_p_qrt_t(beats_list)
    c_pt_waves = np.array(p_waves + t_waves)
    qrs_predicted = self.qrs_kmeans.predict(qrs)
    pt_predicted = self.pt_kmeans.predict(c_pt_waves)
    
    qrs_letters = convert_qrs_predictions_to_letters(qrs_predicted)
    pt_letters = convert_pt_predictions_to_letters(pt_predicted)
    return join_waves_letters_to_words(qrs_letters, pt_letters)

In [4]:
# Word2Vec

In [34]:
def teach_word2vec_or_get_from_path(words, file_path, num_features, use_cached = False):
  """Create and save model to file_path Word2Vec model.
  """
  min_word_count = 1   # Minimum word count
  num_workers = 3       # Number of threads to run in parallel
  context = 10          # Context window size
  downsampling = 1e-3   # Downsample setting for frequent words
  if not (os.path.exists(file_path) and use_cached):
    model = Word2Vec([words,], workers=num_workers,
                     vector_size=num_features, min_count = min_word_count,
                     window = context, sample = downsampling)
    model.save(file_path)
  else:
    print("Using cached Word2Vec from file: ", file_path)
    model = Word2Vec.load(file_path)

  return model

def create_beats_features(words, num_features, word2vec_model):
  review_feature_vecs = np.zeros((len(words),num_features), dtype='float32')

  index2word_set =set(word2vec_model.wv.index_to_key)

  indices = []
  valid_words_count = 0
  for i in range(len(words)):
    cur_word = words[i]
    if cur_word in index2word_set:
      review_feature_vecs[valid_words_count] = word2vec_model.wv[cur_word]
      valid_words_count = valid_words_count + 1
      indices.append(i)

  return indices, review_feature_vecs[:valid_words_count]


In [6]:
# Loading and preparing MIT DB

In [7]:
invalid_beat = [
  "[", "!", "]", "x", "(", ")", "p", "t",
  "u", "`", "'", "^", "|", "~", "+", "s",
  "T", "*", "D", "=", '"', "@"
]

abnormal_beats = [
  "L", "R", "B", "A", "a", "J", "S", "V",
  "r", "F", "e", "j", "n", "E", "/", "f", "Q", "?"
]

def classify_beat(symbol):
  if symbol in abnormal_beats:
    return 1
  elif symbol == "N" or symbol == ".":
    return 0

def define_r_peaks_indices(wave):
  # Defining r-peaks
  fs = 250
  detectors = Detectors(fs)
  r_peaks = detectors.engzee_detector(wave)
  # Amendment for first peak
  r_peaks[0] += 20
  return r_peaks

def define_heartbeat_len(r_peaks):
  distances = []
  for i in range(len(r_peaks)):
    if i + 1 < len(r_peaks):
      distances.append(r_peaks[i + 1] - r_peaks[i])
  distances = np.array(distances)
  heart_beat_len = int(distances.mean())
  return heart_beat_len

def get_sequence(signal, beat_loc, window_sec, fs):
  window_one_side = window_sec * fs
  beat_start = beat_loc - window_one_side
  beat_end = beat_loc + window_one_side
  if beat_end < signal.shape[0]:
    sequence = signal[beat_start:beat_end, 0]
    return sequence
  else:
    return np.array([])
def split_in_beats(wave, heart_beat_len, r_peaks):
  beats = []
  for i in range(len(r_peaks)):
    if i != 0 :
      r_index = r_peaks[i]
      retreat = heart_beat_len // 2
      beats.append(wave[r_index - retreat:r_index + retreat])

  return beats

def download_dataset(out_dir):
  if os.path.isdir(out_dir):
    print('You already have the data')
  else:
    wfdb.dl_database('mitdb', out_dir)

def load(out_dir, record_number):
  download_dataset(out_dir)
  record = wfdb.rdrecord(str(out_dir) + '/' + str(record_number))
  annotation = wfdb.rdann(str(out_dir) + '/' + str(record_number), "atr")
  atr_symbols = annotation.symbol
  atr_samples = annotation.sample
  fs = record.fs
  scaler = StandardScaler()
  signal = scaler.fit_transform(record.p_signal)

  # r_peaks = define_r_peaks_indices(record_wave)
  # heartbeat_len = define_heartbeat_len(r_peaks)
  # beats = split_in_beats(record_wave, heartbeat_len, r_peaks)
  labels = []
  valid_beats = []
  window_sec = 3
  for i, i_sample in enumerate(atr_samples):
    label = classify_beat(atr_symbols[i])
    if label is not None:
      sequence = get_sequence(signal, i_sample, window_sec, fs)
      if sequence.size > 0:
        labels.append(label)
        valid_beats.append(sequence)

  normal_percentage = sum(labels) / len(labels)

  assert len(valid_beats) == len(labels)
  return valid_beats, labels, normal_percentage

In [8]:
def combine_sets_beats_and_features(sets_numbers):
  sets_data = []
  for set_number in sets_numbers:
    print("Loading set: " + str(set_number))
    beats, labels, normal_percentage = load("../mitdb", set_number)
    sets_data.append({
      "beats": beats,
      "labels": labels,
      "normal_percentage": normal_percentage
    })
  return sets_data

def vectorize_using_prepared_word2vec(words, labels, word2vec_model, num_features):
  valid_beats_indices, features = create_beats_features(words, num_features, word2vec_model)

  valid_beats_labels = [labels[i] for i in valid_beats_indices]
  return features, valid_beats_labels

def concatenate_rows(dataframe):
  """

  :param dataframe: 
  :return: dataframe with connected rows
  """
  all_beats = []
  all_labels = []
  for i, row in dataframe.iterrows():
    all_beats.append(row["beats"])
    all_labels.append(row["labels"])
  
  beats = np.concatenate(all_beats).tolist()
  labels = np.concatenate(all_labels).tolist()
  return pd.DataFrame({
    "beats": beats,
    "labels": labels
  })

def get_sets_names(dir_path):
  from os import listdir
  from os.path import isfile, join
  onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]
  def get_file_name(file):
    return file.split(".", 1)[0]

  data = [int(x) for x in list(set(list(map(get_file_name, onlyfiles))))]
  data.sort()
  return data

def load_and_save_set(filename, reload=False):
  if os.path.exists(filename) and not reload:
    print("Using cached dataset")
    return pd.read_pickle(filename)
  else:
    print("Reloading dataset")
    data_frame = pd.DataFrame(combine_sets_beats_and_features(get_sets_names("../mitdb")))
    data_frame.to_pickle(filename)
    return data_frame

def prepare_train_and_test_sets(reload=False, train_path = "train_db_cache.pkl", validation_path = "validation_db_cache.pkl", reload_database_set = False, database_set_path ="database_cache.pkl", sets_count_limit=None):
  if reload or not (os.path.exists(train_path) and os.path.exists(validation_path)):
    print("Loading dataset")
    data_frame = load_and_save_set(database_set_path, reload_database_set)
    if sets_count_limit is not None:
      data_frame = data_frame[:sets_count_limit]
    print("Splitting dataset")
    bins = [0, 0.2, 0.6, 1.0]
    data_frame["bin"] = pd.cut(data_frame['normal_percentage'], bins=bins, labels=False, include_lowest=True)
    train, validation = train_test_split(data_frame, test_size=0.25, stratify=data_frame["bin"], random_state=42)
    
    print("Applying explode to train data")
    train_df = concatenate_rows(train)
    print("Saving train data")
    train_df.to_pickle(train_path)
    
    print("Applying explode to validation data")
    validation_df = concatenate_rows(validation)
    print("Saving validation data")
    validation_df.to_pickle(validation_path)
    
    print("DF to lists conversion")
    return (train_df["beats"].tolist(), train_df["labels"].tolist()), (validation_df["beats"].tolist(), validation_df["labels"].tolist())
  else:
    print("Using cached train&validation datasets")
    train_df = pd.read_pickle(train_path)
    validation_df = pd.read_pickle(validation_path)
    return (train_df["beats"].tolist(), train_df["labels"].tolist()), (validation_df["beats"].tolist(), validation_df["labels"].tolist())

In [42]:
class TestParameters:
  # Limit number of beats in train set (Can invalidate result)
  train_size_limit: int
  # Limit number of beats in validation set (Can invalidate result)
  validation_size_limit: int
  
  # Limit number of sets used in test
  base_data_sets_count_limit: int
  
  # Use cached train&validation sets with names: train_path and validation_path
  use_cache_train_and_validation_sets: bool
  train_path: str
  validation_path: str
  
  reset_beats2words_cache: bool
  beats2words_cache_path: str
  
  reset_word2vec_cache: bool
  word2vec_cache_path: str
  
  
  def __init__(self, train_size_limit=None, validation_size_limit=None, base_data_sets_count_limit=None, use_cache_train_and_validation_sets=True, train_path="train_db_cache.pkl", validation_path="validation_db_cache.pkl", reset_beats2words_cache=True, beats2words_cache_path="beats2words", reset_word2vec_cache=True, word2vec_cache_path="word2vecModel"):
    self.train_size_limit = train_size_limit
    self.validation_size_limit = validation_size_limit
    self.base_data_sets_count_limit= base_data_sets_count_limit
    self.use_cache_train_and_validation_sets = use_cache_train_and_validation_sets
    self.train_path=train_path
    self.validation_path = validation_path
    self.reset_beats2words_cache = reset_beats2words_cache
    self.beats2words_cache_path = beats2words_cache_path
    self.reset_word2vec_cache = reset_word2vec_cache
    self.word2vec_cache_path = word2vec_cache_path
  
  def __str__(self):
    return "{\ntrain_size_limit=" + str(self.train_size_limit)\
           +"\nvalidation_size_limit=" + str(self.validation_size_limit)\
           +"\nbase_data_sets_count_limit=" + str(self.base_data_sets_count_limit)\
           +"\nuse_cache_train_and_validation_sets=" + str(self.use_cache_train_and_validation_sets)\
           +"\ntrain_path=" + str(self.train_path)\
           +"\nvalidation_path=" + str(self.validation_path)\
           +"\nreset_beats2words_cache=" + str(self.reset_beats2words_cache)\
           +"\nbeats2words_cache_path=" + str(self.beats2words_cache_path)\
           +"\nreset_word2vec_cache=" + str(self.reset_word2vec_cache)\
           +"\nword2vec_cache_path=" + str(self.word2vec_cache_path)\
           + "\n}"
  

def prepare_data_for_random_forest(test_parameters):
  """
  
  :param test_parameters: TestParameters
  :return: 
  """
  (train_beats, train_labels), (validation_beats, validation_labels) = prepare_train_and_test_sets(
    reload=not test_parameters.use_cache_train_and_validation_sets,
    train_path=test_parameters.train_path,
    validation_path=test_parameters.validation_path,
    sets_count_limit=test_parameters.base_data_sets_count_limit
  )
  if test_parameters.train_size_limit is not None:
    train_beats = train_beats[:test_parameters.train_size_limit]
    train_labels = train_labels[:test_parameters.train_size_limit]
  if test_parameters.validation_size_limit is not None:
    validation_beats = validation_beats[:test_parameters.validation_size_limit]
    validation_labels = validation_labels[:test_parameters.validation_size_limit]
  
  # Create KMeans model based on train data
  print("Training BeatsToWordsConverter")
  beats2words_converter = BeatsToWordsConverter()
  train_words = beats2words_converter.fit_and_predict_words(train_beats, test_parameters.beats2words_cache_path, test_parameters.reset_beats2words_cache)
  
  # Convert test data into words based on trained BeatsToWordsConverter
  print("Predicting validation words using trained BeatsToWordsConverter")
  validation_words = beats2words_converter.predict_words(validation_beats)

  # Create Word2Vec model based on train data
  print("Training Word2Vec")
  num_features = 300
  word2Vec_model = teach_word2vec_or_get_from_path(train_words, test_parameters.word2vec_cache_path, num_features, not test_parameters.reset_word2vec_cache)

  print("Vectorizing train data")
  # Vectorize train data using Word2Vec
  train_data = vectorize_using_prepared_word2vec(train_words, train_labels, word2Vec_model, num_features)

  print("Vectorizing test data")
  # Vectorize test data using trained Word2Vec
  test_data = vectorize_using_prepared_word2vec(validation_words, validation_labels, word2Vec_model, num_features)
  return train_data, test_data

def execute_random_forest_test(test_parameters, comment):
  """
  
  :param test_parameters: TestParameters
  :param comment: 
  :return: 
  """
  test_start_time = datetime.now()
  
  (train_x, train_y), (validation_x, validation_y) = prepare_data_for_random_forest(test_parameters)

  print("Training random forest")
  forest = RandomForestClassifier(n_estimators = 100, verbose=True, n_jobs=4)
  forest = forest.fit(train_x, train_y)

  print("Predicting using random forest")
  result = forest.predict(validation_x)

  print("Creating classification report")
  repord_id = uuid.uuid1()
  report = str(classification_report(validation_y, result))
  print("Report id: " + str(repord_id))
  print(report)
  f = open("results.txt", "a")
  
  now = datetime.now()
  f.write("===========================================\n")
  f.write("Report id: " + str(repord_id) + "\n")
  f.write("Report start date: " + test_start_time.strftime("%d.%m.%Y %H:%M:%S") + "\n")
  f.write("Report end date: " + now.strftime("%d.%m.%Y %H:%M:%S") + "\n")
  f.write("Test parameters:\n" + str(test_parameters) + "\n")
  f.write("Actual train size: " + str(len(train_x)) + "\n")
  f.write("Actual validation size: " + str(len(validation_x)) + "\n")
  f.write("Comment: " + comment + "\n")
  f.write(report)
  f.write("\n===========================================")
  f.write("\n\n")
  f.close()
  #os.system("shutdown /s /t 1")

In [50]:
execute_random_forest_test(TestParameters(
  train_size_limit=None,
  validation_size_limit=None,
  base_data_sets_count_limit=20,
  
  use_cache_train_and_validation_sets=True,
  train_path="short20_train_db_cache.pkl",
  validation_path="short20_validation_db_cache.pkl",
  
  reset_beats2words_cache=True,
  beats2words_cache_path="beats2words_20",
  reset_word2vec_cache=True,
  word2vec_cache_path="word2vec_20.pkl"
), "Test with first 6 datasets with cached from 5 datasets word2vec and kmeans")


Using cached train&validation datasets
Training BeatsToWordsConverter
Fitting QRS KMeans
Fitting PT KMeans
Saving BeatsToWordsConverter to cache:  beats2words_20
Predicting validation words using trained BeatsToWordsConverter
Training Word2Vec
Vectorizing train data
Vectorizing test data
Training random forest
Predicting using random forest
Creating classification report
Report id: bc8936e4-29bf-11ec-bde0-ec8eb50e3be9
              precision    recall  f1-score   support

           0       0.91      0.60      0.72      6841
           1       0.38      0.81      0.52      2120

    accuracy                           0.65      8961
   macro avg       0.65      0.70      0.62      8961
weighted avg       0.79      0.65      0.67      8961



[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   13.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.6s finished
